In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Cargar los datos
import pandas as pd
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label','text'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label','text'])

# Convertir las etiquetas a números
train_data['label_num'] = train_data.label.map({'ham':0, 'spam':1})
test_data['label_num'] = test_data.label.map({'ham':0, 'spam':1})

# Tokenizar y vectorizar el texto
from tensorflow import keras
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_data.text)
x_train = tokenizer.texts_to_sequences(train_data.text)
x_test = tokenizer.texts_to_sequences(test_data.text)

# Rellenar secuencias para longitud fija
x_train = keras.preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=100)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, padding='post', maxlen=100)

y_train = train_data.label_num.values
y_test = test_data.label_num.values

In [ ]:
# Crear modelo de red neuronal
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=100),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar modelo
model.fit(x_train, y_train, epochs=10, batch_size=512, validation_data=(x_test, y_test), verbose=2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    padded = keras.preprocessing.sequence.pad_sequences(sequence, padding='post', maxlen=100)
    prob = model.predict(padded)[0][0]
    label = 'ham' if prob < 0.5 else 'spam'
    return [prob, label]

# Prueba de la función
print(predict_message("how are you doing today?"))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
